In [ ]:
import os
import pandas as pd
import numpy as np

# Ruta raíz
root_dir = "Test/experimento_red&green"
nombre_metricas = "metricas_por_k.xlsx"
metricas_objetivo = ["accuracy", "f1_score", "roc_auc"]

# Valores esperados para MultiIndex
anomalies = ["0.01", "0.05", "0.1", "0.2", "0.5", "0.6"]
variantes = ["dataset_embeddings_encoder_resnet.csv", "dataset_embeddings_encoder.csv", "embeddings.csv", "tus_embeddings.csv"]
codificaciones=["Autoencoder RESNET","Autoencoder simple","Modelo preentrenado","Flatten"]
index_esperado = pd.MultiIndex.from_product([anomalies, codificaciones], names=["Anomaly", "Codificación"])
num_filas_esperadas = len(index_esperado)  # 6 * 4 = 24
print(num_filas_esperadas)
# Diccionario final
matrices_resultado = {}

# Recorrer métodos
for metodo in sorted(os.listdir(root_dir)):
    metodo_path = os.path.join(root_dir, metodo)
    if not os.path.isdir(metodo_path):
        continue

    print(f"Procesando método: {metodo}")
    matrices_resultado[metodo] = {metrica: [] for metrica in metricas_objetivo}

    for anomaly_percentage in anomalies:
        anomaly_path = os.path.join(metodo_path, anomaly_percentage)
        if not os.path.isdir(anomaly_path):
            continue
        
        if metodo=="reconstrucción" or metodo=="semilla_aleatoria" or metodo=="ZeroR":
            df = pd.read_excel(os.path.join(anomaly_path, nombre_metricas))
            for metrica in metricas_objetivo:
                if metrica in df.columns:
                    valor = df[metrica].values
                    matrices_resultado[metodo][metrica].append(valor)
                else:
                    print(f"⚠️ Métrica '{metrica}' no encontrada en {variante_path}")
          
        
        for variante in variantes:
            variante_path = os.path.join(anomaly_path, variante)
            if not os.path.isdir(variante_path):
                continue

            try:
                df = pd.read_excel(os.path.join(variante_path, nombre_metricas))
                for metrica in metricas_objetivo:
                    if metrica in df.columns:
                        valor = df[metrica].values
                        matrices_resultado[metodo][metrica].append(valor)
                    else:
                        print(f"⚠️ Métrica '{metrica}' no encontrada en {variante_path}")
            except Exception as e:
                print(f"❌ Error leyendo {variante_path}: {e}")

        # Convertir a DataFrame con MultiIndex si es posible
    for metrica in metricas_objetivo:
        filas = matrices_resultado[metodo][metrica]
        
        if not filas:
            matrices_resultado[metodo][metrica] = None
            print(f"❌ Sin datos para {metodo} - {metrica}")
            continue

        try:
            # Rellenar con nan si hay longitudes distintas
            max_cols = max(len(f) for f in filas)
            m = np.array([
                np.pad(f.astype(float), (0, max_cols - len(f)), constant_values=np.nan)
                for f in filas
                        ])
            
            if m.shape[0] == num_filas_esperadas:
                df_resultado = pd.DataFrame(m, index=index_esperado)
            else:
                df_resultado = pd.DataFrame(m)
                print(f"⚠️ Método '{metodo}', métrica '{metrica}' tiene {m.shape[0]} filas, se usa índice simple")

            matrices_resultado[metodo][metrica] = df_resultado
            print(f"✅ Guardado: {metodo} - {metrica} -> shape {df_resultado.shape}")

        except Exception as e:
            print(f"❌ Error al procesar {metodo} - {metrica}: {e}")
            matrices_resultado[metodo][metrica] = None

print(matrices_resultado["GDA"])


# Guardar un archivo .xlsx por cada método con una hoja por métrica
output_dir = "resultados_excel_red&green"
os.makedirs(output_dir, exist_ok=True)

for metodo, metricas_dict in matrices_resultado.items():
    output_path = os.path.join(output_dir, f"{metodo}.xlsx")
    with pd.ExcelWriter(output_path) as writer:
        for metrica, df in metricas_dict.items():
            if isinstance(df, pd.DataFrame):
                nombre_hoja = metrica[:31]  # Limita nombre a 31 caracteres (máximo en Excel)
                df.to_excel(writer, sheet_name=nombre_hoja)
                print(f"💾 Guardado '{metodo}' - '{metrica}' en {output_path}")
            else:
                print(f"⚠️ No se guarda '{metodo}' - '{metrica}': no es DataFrame")




24
Procesando método: 0.01
❌ Sin datos para 0.01 - accuracy
❌ Sin datos para 0.01 - f1_score
❌ Sin datos para 0.01 - roc_auc
Procesando método: 0.05
❌ Sin datos para 0.05 - accuracy
❌ Sin datos para 0.05 - f1_score
❌ Sin datos para 0.05 - roc_auc
Procesando método: 0.1
❌ Sin datos para 0.1 - accuracy
❌ Sin datos para 0.1 - f1_score
❌ Sin datos para 0.1 - roc_auc
Procesando método: 0.2
❌ Sin datos para 0.2 - accuracy
❌ Sin datos para 0.2 - f1_score
❌ Sin datos para 0.2 - roc_auc
Procesando método: 0.5
❌ Sin datos para 0.5 - accuracy
❌ Sin datos para 0.5 - f1_score
❌ Sin datos para 0.5 - roc_auc
Procesando método: 0.6
❌ Sin datos para 0.6 - accuracy
❌ Sin datos para 0.6 - f1_score
❌ Sin datos para 0.6 - roc_auc


KeyError: 'GDA'